# Example filtering enriched metadata from KNTraP
KNTraP candidate light-curves and positional information is used to:
1. Positional cross-match with catalogues: to provide information on the actual source if catalogued or its environment (aka galaxy close-by)
2. Light-curve processing: to obtain magnitude increase/decrease rates and colour


These "metadata" can be then used to discard known stars and select most promising transients.


This notebook allows you to create your own selection.

In [58]:
import pandas as pd
import numpy as np

Beware! as of March 9th 2022 the enriched metadata is saved as a pickle file. This allows numpy arrays to be saved properly

df = pd.read_pickle('./Fink_outputs/353A_tmpl.pickle')

What are the available metadata?

| key | Content | Type |
| --- | --- | --- |
|id| KNTraP id, beware this is run dependent| int |
|ra| Candidate ra forced phot | float |
|dec | Candidate dec forced phot | float |
|max_mag_i| Maximum magnitude in forced photometry i band|float |
|max_mag_g| Maximum magnitude in forced photometry g band|float |
|min_mag_i| Minimum magnitude in forced photometry i band|float |
|min_mag_g| Minimum magnitude in forced photometry g band|float |
|mean_mag_i| Mean magnitude in forced photometry i band|float |
|mean_mag_g| Mean magnitude in forced photometry g band|float |
|std_mag_i| Dispersion magnitude in forced photometry i band|float |
|std_mag_g| Dispersion magnitude in forced photometry g band|float |
|dmag_i| Delta Magnitude in i band | np.array |
|dmag_g| Delta Magnitude in g band | np.array |
|dmag_rate_i| Delta Magnitude rate (per day) in i band | np.array |
|dmag_rate_g| Delta Magnitude rate (per day) in g band | np.array |
|color| i-g color if measurements on same night (how red it is) | np.array |
|color_avg| Average color complete light-curve forced phot | float |
|ndet| Numbers of detections in forced phot | int |
|two_mags_gt_225| Are two measured magnitudes above 22.5? (useful for shallow fields)| Boolean |
|two_mags_gt_235| Are two measured magnitudes above 23.5? (useful for deep fields) | Boolean |
|ra_unforced| Candidate ra unforced phot | float | 
|dec_unforced| Candidate dec unforced phot | float | 
|max_mag_i_unforced| Same as above unforced | float |
|max_mag_g_unforced| | |
|min_mag_i_unforced| | |
|min_mag_g_unforced| | |
|mean_mag_i_unforced| | |
|mean_mag_g_unforced| | |
|std_mag_i_unforced| | |
|std_mag_g_unforced| | |
|dmag_i_unforced| | |
|dmag_g_unforced| | |
|dmag_rate_i_unforced| | |
|dmag_rate_g_unforced| | |
|color_unforced| | |
|color_avg_unforced| | |
|ndet_unforced| | |
|two_mags_gt_225_unforced| | |
|two_mags_gt_235_unforced| | |
|simbad_type| Simbad positional cross match type 5''| string |
|simbad_ctlg| Simbad positional cross match catalogue 5'' | string |
|simbad_sptype| Simbad positional cross match sptype 5'' | string |
|simbad_redshift| Simbad positional cross match redshift | float |
|gaia_DR2_source| Gaia DR2 positional cross match 2'' source | string |
|gaia_DR2_ra| Gaia DR2 positional 2'' cross match ra | float |
|gaia_DR2_dec| Gaia DR2 positional 2'' cross match dec | float |
|gaia_DR2_parallax| Gaia DR2 positional 2'' cross match parallax | float |
|gaia_DR2_parallaxerr| Gaia DR2 positional 2'' cross match parallax error | float |
|gaia_DR2_gmag| Gaia DR2 positional 2'' cross match magnitude in g | float |
|gaia_DR2_angdist| Gaia DR2 positional 2'' cross match angular distance | float |
|gaia_eDR3_source| idem but for early data release 3|
|gaia_eDR3_ra| |
|gaia_eDR3_dec| |
|gaia_eDR3_parallax| |
|gaia_eDR3_parallaxerr| |
|gaia_eDR3_gmag| |
|gaia_eDR3_angdist| |
|USNO_source| USNO (stars) positional cross match 2'' source | string |
|USNO_angdist| USNO (stars) positional cross match 2'' angular distance | float |

# Define your filtering
Now you can define which candidates you are interested in inspecting.

A basic example would be querying for transients with at least 3 detections in forced photometry and two measured magnitudes greater than 23.5

In [63]:
my_query_string = ("ndet>=3 & two_mags_gt_235==True")
df_out = df.query(my_query_string)

In [68]:
print(f"Our original list of {len(df)} candidates, has now been reduced to {len(df_out)}")

Our original list of 171 candidates, has now been reduced to 89


Now you can save this list for visualization (uncomment to do so)

In [72]:
# df_out.to_csv('my_filtering',sep=";",index=False)

# More advanced queries

#### A note about SIMBAD cross-matches

SIMBAD type will return "Unknown" if no matching source found, "Fail" if service had an issue cross-matching it.

If looking for extra-galactic objects it may also return a host-galaxy cross-match as the closest source. Suggest to keep these sources for the filtering.

In [61]:
list_simbad_galaxies = [
    "galaxy",
    "Galaxy",
    "EmG",
    "Seyfert",
    "Seyfert_1",
    "Seyfert_2",
    "BlueCompG",
    "StarburstG",
    "LSB_G",
    "HII_G",
    "High_z_G",
    "GinPair",
    "GinGroup",
    "BClG",
    "GinCl",
    "PartofG",
]
keep_cds = ["Unknown", "Transient", "Fail"] + list_simbad_galaxies

#### Adding a requirement for delta mag array

You could want to check if there is at least one measurement with delta mag in the i band > 0.3, to do so
- create a column True or False depending on my requirement for delta mag

In [74]:
df = df.assign(mymagrequirement=df.dmag_i.apply(lambda x: np.any(x>0.3)))

In [75]:
my_query_string = ("ndet>=3 & two_mags_gt_235==True & mymagrequirement==True")

In [76]:
df.query(my_query_string)

,id,ra,dec,max_mag_i,max_mag_g,min_mag_i,min_mag_g,mean_mag_i,mean_mag_g,std_mag_i,...,gaia_eDR3_source,gaia_eDR3_ra,gaia_eDR3_dec,gaia_eDR3_parallax,gaia_eDR3_parallaxerr,gaia_eDR3_gmag,gaia_eDR3_angdist,USNO_source,USNO_angdist,mymagrequirement
3,353A_50.8_cand715,60.803379,-11.768675,20.240,22.421,21.854,22.421,21.057250,22.421000,0.667708,...,3189920456108861312,60.80333259738,-11.76860807751,1.2143,0.0872,17.280184,0.378151,0750-00889878,0.548233,True
6,353A_57.8_cand678,59.878162,-12.189603,19.669,19.897,21.651,20.379,20.885333,20.117333,1.065085,...,3189985327294831872,59.87826301517,-12.18971173869,0.7415,0.0392,15.823053,0.535739,0750-00874605,0.837959,True
22,353A_51.8_cand718,59.403075,-11.952289,21.747,NaN,22.075,NaN,21.902500,NaN,0.178468,...,3190053939396273408,59.40308966260,-11.95227462467,Unknown,Unknown,20.753521,0.072827,0750-00867257,0.483451,True
36,353A_20.8_cand19,59.687954,-11.207792,21.720,22.290,22.684,22.290,22.156000,22.290000,0.445765,...,3193199367284916480,59.68814129302,-11.20787309966,-0.3704,0.4155,19.725742,0.663150,Unknown,Unknown,True
41,353A_18.8_cand187,60.922329,-11.037994,19.744,NaN,20.868,NaN,20.218000,NaN,0.541745,...,3190544497677021184,60.92227687017,-11.03822525200,0.6564,0.0365,15.610655,0.815106,0750-00891830,0.658813,True
43,353A_18.8_cand1286,60.945350,-10.973522,22.348,21.621,23.167,22.196,22.785000,21.977250,0.412261,...,3190592047258476800,60.94535002194,-10.97351542636,-0.1464,0.3225,19.288380,0.028724,0750-00892227,0.407141,True
45,353A_18.8_cand748,60.830079,-10.969653,19.588,20.399,21.831,21.949,20.407333,21.341000,1.237609,...,3190591330000348416,60.83005696503,-10.96984217789,1.0795,0.0320,15.233547,0.825692,0750-00890267,1.209863,True
49,353A_27.8_cand442,59.761462,-11.328167,21.078,NaN,22.407,NaN,21.582667,NaN,0.719861,...,3193194526858845696,59.76142301592,-11.32843644295,0.8164,0.0516,16.510395,0.755758,0750-00872802,0.278438,True
56,353A_27.8_cand1132,59.638521,-11.359683,21.614,21.105,22.268,21.194,21.882250,21.138250,0.317163,...,3193103297456830720,59.63847658595,-11.35970436185,-0.0877,0.1435,18.156254,0.174210,0750-00870802,0.148639,True
62,353A_7.8_cand1290,60.392267,-10.632914,20.856,20.610,21.935,20.963,21.395500,20.834250,0.762968,...,3193621171728321408,60.39240582606,-10.63287184502,Unknown,Unknown,16.702703,0.515111,0750-00883300,0.241553,True
